### 객체 탐지(object Detection)용 오픈 소스인 Yolov5를 사용해서 객체를 검출해보자!
- 객체 영역 판단과 객체 탐지를 동시에 진행하는 One stage detection 방식
- 기존 CNN을 활용한 이미지 분류는 이미지 전체 범위를 하나의 클래스(정답)로 표시한다면, Yolo는 이미지 내에 특정한 위치와 객체 탐지를 진행한 후 이를 설정된 클래스와 바운딩 박스(Bbox)로 표시함

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 데이터 수집
- CNN처럼 이미지와 정답(숫자)만 있으면 되는 것이 아니라 이미지 내에 정답이 될 객체들을 일일이 분리해서 정답으로 설정해줘야함(정답에 대한 좌표값과 정보들이 필요)
- roboflow 사이트에서 라벨링 된 권총 데이터를 사용해보자!
- 자신만의 데이터로 학습을 시키고 싶다면 데이터 라벨링 작업을 수작업ㅇ로 진행해서 정답을 만들어줘야 하며 이를 도와주는 Tool들을 사용할 수 있음(labeling, roboflow의 annotate 등)

In [2]:
!mkdir dataset

!curl -L "https://public.roboflow.com/ds/hbY4OeHqc8?key=qiLfibSJP0" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: export/images/armas (693)_jpg.rf.539e7f818abfba8b13efc60c3eaa4b09.jpg  
 extracting: export/images/armas (1623)_jpg.rf.54322b09fff9789a2617f07a19f8f531.jpg  
 extracting: export/images/armas (772)_jpg.rf.5335483aeff9c87fd46a3a12e5a75f4d.jpg  
 extracting: export/images/armas (1101)_jpg.rf.53cb10cc7705158adb60227aa482711d.jpg  
 extracting: export/images/armas (623)_jpg.rf.53ee22796f191ca0eaf6124f4e133f5f.jpg  
 extracting: export/images/armas (2278)_jpg.rf.535d90355a01edf613c11d1d6168de3e.jpg  
 extracting: export/images/armas (2254)_jpg.rf.5451063a5496a056897d719fb4ccdc83.jpg  
 extracting: export/images/armas (1306)_jpg.rf.549461e692756d4a158af908e7eb8843.jpg  
 extracting: export/images/armas (2797)_jpg.rf.5518ea2e6baa681a051807d54dae5d9d.jpg  
 extracting: export/images/armas (1947)_jpg.rf.54cd92bf56c54ddc500de583356367a2.jpg  
 extracting: export/images/armas (304)_jpg.rf.54d3beb055395743c1d0df76e3214b6f.jpg  
 extracting: export/im

#### export 폴더의 images에는 실제 이미지 데이터, labels에는 권총이라는 객체와 박스의 좌표값이 표시되어 있음
- labels의 첫번째 0 값은 권총이라는 class(정답)를 의미, class가 여러개라면 0, 1, 2 이런식으로 라벨링 됨
- labels의 두번째부터 다섯번째 값은 정답 Bbox의 x, y, w, h 값(박스의 중앙좌표, 너비, 높이)

### 2. Yolo 모델 환경설정 및 사전작업
- Yolov5 깃 허브 주소 : https://github.com/ultralytics/yolov5

In [3]:
%cd /content

/content


In [4]:
# 깃 허브 코드 클론(Yolov5 공식 깃허브 소스 복제해오기!)
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 15937, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 15937 (delta 26), reused 29 (delta 14), pack-reused 15880
Receiving objects: 100% (15937/15937), 14.66 MiB | 13.70 MiB/s, done.
Resolving deltas: 100% (10921/10921), done.


- Yolov5 구동에 필요한 라이브러리 전부 install

In [5]:
%cd /content/yolov5

/content/yolov5


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.8/613.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


- data.yaml : 데이터에 대한 정보가 담겨져 있는 파일로 Yolo 학습시 yaml 파일에 있는 데이터의 정보를 바탕으로 모델이 학습하게 됨

In [7]:
# cat : 파일의 모든 내용 출력
%cat /content/dataset/data.yaml

# train, val(test로 바꿔도 상관없음) 각각의 데이터들이 있는 경로 설정
# nc(number of classes) : 클래스 종류의 개수
# names : 해당 클래스들의 이름

cat: /content/dataset/data.yaml: No such file or directory


In [8]:
%cd /content

/content


In [12]:
# glob : 많은 양의 파일들을 한번에 처리할 때 사용하는 라이브러리
from glob import glob
img_list = glob('/content/dataset/export/images/*.jpg')
len(img_list)

2971

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# 데이터 자체를 분리한 것이 아닌 데이터가 있는 경로를 분리
train_img_list, valid_img_list = train_test_split(img_list, test_size=0.2, random_state=11)

print(len(train_img_list))
print(len(valid_img_list))

2376
595


In [15]:
# 모든 이미지들의 경로들을 하나의 txt 파일에 담아주기(train, val 각각)

# with open : 파일에 접근(없는 파일명이라면 새롭게 생성됨)
# w : 읽기+쓰기 가능
with open('/content/dataset/train.txt', 'w') as f:
  # 만들어진 txt 파일에 이미지 경로 입력
  f.write('\n'.join(train_img_list)+'\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(valid_img_list)+'\n')


- data.yaml 파일의 train,val 경로에 우리 데이터의 경로가 있는 txt 파일을 넣기

In [16]:
import yaml
with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)
data

{'train': '../train/images',
 'val': '../valid/images',
 'nc': 1,
 'names': ['pistol']}

In [17]:
print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['pistol']}


In [ ]:
# 딕셔너리의 value값 변경하기
data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

In [ ]:
wit